## Import Libraries

In [109]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from itertools import combinations
from tqdm.notebook import tqdm
import time

#import os
#os.chdir('/Users/yvadalia/Library/CloudStorage/OneDrive-Chewy.com,LLC/Project/pick_planning')

#from experiments.utils.connection import create_con
#con=create_con()
import vertica_python
from cryptography.fernet import Fernet


In [110]:

#Fetch encrypted credentials for Vertica EDW connection
def read_secrets(file_path):
    with open(file_path, 'r') as file:
        key = file.readline().strip()                    # Read the key
        encrypted_credentials = file.readline().strip()  # Read the encrypted credentials
    return key, encrypted_credentials

#Decrypt credentials using key
def decrypt_credentials(key, encrypted_credentials):
    cipher_suite = Fernet(key)
    decrypted_credentials = cipher_suite.decrypt(encrypted_credentials.encode())
    return decrypted_credentials.decode()

#File Path to credentials
file_path = 'C:/Users/abhatt/cred.txt'

# Read the key and encrypted credentials
key, encrypted_credentials = read_secrets(file_path)

# Decrypt the credentials
decrypted_credentials = decrypt_credentials(key, encrypted_credentials)
conn_info = {'host': 'bidb.chewy.local', 'port': 5433, 'database': 'bidb', 'user': 'abhatt', 'password':decrypted_credentials ,'use_prepared_statements':False, 'autocommit': True}
alpha = 0.95


## Import Data

In [111]:
%%time
with vertica_python.connect(**conn_info) as connection:
    cur = connection.cursor()
    order_data_query = '''
-- Fetch containers AutoBatch date
with AutoBatch_release as (
select distinct
         tcpd.wh_id,      
         tcpd.container_id,
         tcp.created_date as autobatched_date
from aad.t_container_print_detail tcpd
left join aad.t_container_print tcp on tcpd.wh_id = tcp.wh_id and tcpd.batch_id = tcp.batch_id
where 
tcpd.wh_id = 'AVP1'
and tcp.released_date::date between '2025-03-06 14:00:00'::date - 4 and '2025-03-06 14:00:00'::date + 2 -- cast a wider net for dates
)
-- Container data for AB simulation arrival 
, final as 
(
        (
        select 
                 pc.container_id as OrderID,
                 (pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York')::date as Arrival_Date, --Convert to local time
                 (pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York') as Arrive_Date,
                 HOUR(pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York') as Arrive_Hour,
                 CASE WHEN MINUTE(pc.arrive_date) > 5 THEN HOUR(pc.arrive_date) + 1 ELSE HOUR(pc.arrive_date) END as Available_To_Batch, 
                 CAST((LEFT(CAST(pc.promised_date as VARCHAR),11)||LEFT(CAST(pc.cutoff_time as VARCHAR),2)||':'||RIGHT(CAST(pc.cutoff_time as VARCHAR),2)||':00') as timestamp) 
                        AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York' as Cut_DateTime,
                 pd.item_number as SKU, 
                 uom.uom_weight,
                 pd.planned_quantity as Pick_Quantity, 
                 pd.pick_location as Pick_Location,
                 right(loc.print_zone,2)::numeric(1,0) as Print_Zone,
                 loc.aisle_sequence as Pick_Aisle,
                 loc.picking_flow_as_int as Picking_Flow_as_int,
                0 as BatchedFlag,
                ' ' as BatchID
        from aad.t_pick_container as pc
        left join aad.t_pick_detail as pd
        on pd.wh_id = pc.wh_id and pc.container_id = pd.container_id
        left Join aad.t_location as loc
        on loc.wh_id = pc.wh_id and loc.location_id = pd.pick_location
        left Join aad.t_zone_loca as z
        on z.wh_id = pc.wh_id and z.location_id = pd.pick_location
        left join aad.t_item_uom as uom 
        on uom.wh_id = pc.wh_id and uom.item_number = pd.item_number
        where pd.wh_id = 'AVP1'
        and pc.arrive_date between '2025-03-06 14:00:00' and '2025-03-06 14:29:00'
        and pc.status in ('SHIPPED')
        and distinct_items > 1
        and z.zone = 'GROUND'
        )
        union 
        -- Containers charged but not batched before selected simulation start time
        (
        select 
                  pc.container_id as OrderID,
                 (pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York')::date as Arrival_Date, --Convert to local time
                 (pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York') as Arrive_Date,
                 HOUR(pc.arrive_date  AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York') as Arrive_Hour,
                 CASE WHEN MINUTE(pc.arrive_date) > 5 THEN HOUR(pc.arrive_date) + 1 ELSE HOUR(pc.arrive_date) END as Available_To_Batch, 
                 CAST((LEFT(CAST(pc.promised_date as VARCHAR),11)||LEFT(CAST(pc.cutoff_time as VARCHAR),2)||':'||RIGHT(CAST(pc.cutoff_time as VARCHAR),2)||':00') as timestamp) 
                        AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York' as Cut_DateTime,
                 pd.item_number as SKU, 
                 uom.uom_weight,
                 pd.planned_quantity as Pick_Quantity, 
                 pd.pick_location as Pick_Location,
                 right(loc.print_zone,2)::numeric(1,0) as Print_Zone,
                 loc.aisle_sequence as Pick_Aisle,
                 loc.picking_flow_as_int as Picking_Flow_as_int,
                0 as BatchedFlag,
                ' ' as BatchID
        from aad.t_pick_container as pc
        left join aad.t_pick_detail as pd
        on pd.wh_id = pc.wh_id and pc.container_id = pd.container_id
        left Join aad.t_location as loc
        on loc.wh_id = pc.wh_id and loc.location_id = pd.pick_location
        left Join aad.t_zone_loca as z
        on z.wh_id = pc.wh_id and z.location_id = pd.pick_location
        left join aad.t_item_uom as uom 
        on uom.wh_id = pc.wh_id and uom.item_number = pd.item_number
        left join AutoBatch_release as ar
        on ar.container_id = pc.container_id
        where pd.wh_id = 'AVP1'
        and pc.arrive_date < '2025-03-06 14:00:00'
        and ar.autobatched_date > '2025-03-06 14:00:00'
        and pc.status in ('SHIPPED')
        and distinct_items > 1
        and z.zone = 'GROUND'
        )
)
select * 
from final 
order by 2,3,4,1,8;
'''

#order_data = pd.read_excel("Simulation_Inputs_arrive_date_20240205_to_20240211.xlsx")
    order_data = pd.read_sql(order_data_query, connection)
    
#order_data = pd.read_csv("05_01_05_15/InputData/AVP1_Slotbook_Order_Data_06_09_06_15_MFP_Heat_Dist_updated.csv")


CPU times: total: 172 ms
Wall time: 17.2 s


In [112]:
# Read container_assignments.csv
container_assignments = pd.read_csv('container_assignments.csv')

# Filter order_data to keep only rows where OrderID is in container_id from container_assignments
order_data = order_data[order_data['OrderID'].astype(str).isin(container_assignments['ContainerID'].astype(str))]

# Show total unique OrderID in the filtered DataFrame
total_unique_order_ids = order_data['OrderID'].nunique()
print(f"Total unique OrderID in the filtered DataFrame: {total_unique_order_ids}")


Total unique OrderID in the filtered DataFrame: 1380


In [113]:
#order_data.rename({'container_id':'OrderID','arrive_date':'Arrive_Date','item_number':'SKU','planned_quantity':'Pick_Quantity','pick_location':'Pick_Location', 'aisle_sequence':'Pick_Aisle'}, axis=1, inplace=True)

order_data['BatchID'] = np.nan
order_data['BatchID'] = order_data['BatchID'].astype('object')


In [114]:
#Fix Datatypes
order_data['OrderID'] = order_data['OrderID'].astype('int32')
order_data['SKU'] = order_data['SKU'].astype('int32')
order_data['Pick_Aisle'] = order_data['Pick_Aisle'].astype('int32')
order_data['Pick_Quantity'] = order_data['Pick_Quantity'].astype('int32')
order_data['Arrive_Hour'] = order_data['Arrive_Hour'].astype('int32')
order_data['Cut_DateTime'] = pd.to_datetime(order_data['Cut_DateTime'])
order_data['Arrive_Date'] = pd.to_datetime(order_data['Arrive_Date'])
order_data['Arrive_Date_Hour'] = pd.to_datetime(order_data['Arrive_Date'].astype(str) + ' ' + order_data['Arrive_Hour'].astype(str) + ':00:00')
# order_data['Arrive_Date_Hour'] = pd.to_datetime(order_data['Arrive_Date'] + ' ' + order_data['Arrive_Hour'].astype(str) + ':00:00', format='%m/%d/%Y %H:%M:%S')
order_data['Available_To_Batch_Date_Hour'] = pd.to_datetime(order_data.apply(lambda x: str((x['Arrive_Date'] + pd.DateOffset(days=1)).date())+' '+str(0) + ':00:00' if x['Available_To_Batch']==24 else str(x['Arrive_Date'].date())+' '+str(x['Available_To_Batch']) + ':00:00', axis=1))
order_data['Print_Zone'] = order_data['Print_Zone'].astype('int32')
order_data['BatchedFlag'] = 0 
order_data['BatchedFlag'] = order_data['BatchedFlag'].astype('int32')
order_data['BatchID'] = order_data['BatchID'].astype('object')  # or .astype('str') if it's always a string
order_data['Rule'] = np.nan
order_data['Rule'] = order_data['Rule'].astype('str')
order_data['Picking_Flow_as_int'] = order_data['Picking_Flow_as_int'].astype('int32')
order_data['Rule_Desc'] = np.nan
order_data['Rule_Desc'] = order_data['Rule_Desc'].astype('str')
order_data['Batch_Date_Hour'] = None

output_order_data = order_data.copy()

# Create Order Master DataFrame
agg_funcs = {
    'OrderID': 'size', 'SKU': 'nunique', 'Pick_Location': 'nunique', 
    'Pick_Aisle': ['nunique', 'min', 'max'], 'Arrive_Date': 'first', 
    'Arrive_Hour': 'first', 'Cut_DateTime': 'first', 'Available_To_Batch': 'first'
}
order_master = order_data.groupby('OrderID').agg(agg_funcs)
order_master.columns = [
    'Total_Lines', 'Distinct_SKUs', 'Distinct_Pick_Locations', 
    'Distinct_Pick_Aisles', 'Min_Pick_Aisle', 'Max_Pick_Aisle',
    'Arrive_Date', 'Arrive_Hour', 'Cut_DateTime', 'Available_To_Batch'
]

order_master.reset_index(inplace=True)
order_master['Total_Pick_Quantity'] = order_data.groupby('OrderID')['Pick_Quantity'].sum().reset_index(drop=True)
order_master['BatchedFlag'] = 0
order_master['BatchID'] = None
order_master['BatchRule'] = None
order_master['BatchDate'] = None
order_master['BatchHour'] = None
order_master['Batch_Date_Hour'] = None
#order_master['Arrive_Date_Hour'] = pd.to_datetime(order_master['Arrive_Date'].astype(str) + ' ' + order_master['Arrive_Hour'].astype(str) + ':00:00')
order_master['Rule'] = np.nan
order_master['Rule'] = order_master['Rule'].astype('str')
# order_master['Arrive_Date_Hour'] = pd.to_datetime(order_master['Arrive_Date'].astype(str) + ' ' + order_master['Arrive_Hour'].astype(str) + ':00:00', format='%m/%d/%Y %H:%M:%S')
order_master['Arrive_Date_Hour'] = pd.to_datetime(order_master['Arrive_Date'].astype(str) + ' ' + order_master['Arrive_Hour'].astype(str) + ':00:00')
order_master['Available_To_Batch_Date_Hour'] = pd.to_datetime(order_master.apply(lambda x: str((x['Arrive_Date'] + pd.DateOffset(days=1)).date())+' '+str(0) + ':00:00' if x['Available_To_Batch']==24 else str(x['Arrive_Date'].date())+' '+str(x['Available_To_Batch']) + ':00:00', axis=1))

order_master['Rule_Description'] = np.nan
order_master['Rule_Description'] = order_master['Rule_Description'].astype('str')

# Initialize DataFrame for Hourly Statistics
hourly_stats = pd.DataFrame(columns=[
    'Date', 'Hour', 'Date_Hour', 'Unique_Batch_ID_Count', 
    #'Unique_Order_Count',
    'Target_Containers', 'Containers_Batched', 'Sum_of_Units_Batched'
])


In [115]:
# # Import Hourly Container Targets
with vertica_python.connect(**conn_info) as connection:
    cur = connection.cursor()
    hourly_target_query = '''
with PreQ as (
        select (log_datetime AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York')::date as batch_date,
               hour(log_datetime AT TIME ZONE 'America/New_York' AT TIME ZONE 'America/New_York') as batch_hour, *
        from aad.t_auto_batching_release_log
        where wh_id = 'AVP1'
        and log_datetime between '2025-03-06 14:00:00' and '2025-03-06 15:00:00'
        and profile_name = 'All'
        and release_now = 'YES'
        order by log_datetime::date, hour(log_datetime), rule_priority
        
)
select wh_id,
       batch_date,
       batch_hour,
       sum(batches_released), 
       13300 as containers_target,
       sum(containers_released) as containers_released
from PreQ
group by 1,2,3
order by 2,3
'''


    container_targets_df = pd.read_sql(hourly_target_query, connection)



In [116]:
# # Update 'Date_Hour' to reflect the correct hour
container_targets_df['Date_Hour'] = pd.to_datetime(container_targets_df['batch_date'].astype(str) + ' ' + container_targets_df['batch_hour'].astype(str) + ':00:00')
container_targets_df.rename({'containers_target':'Containers_Target'}, axis=1, inplace=True)


## AutoBatch Logic

## Rule 1 - Single Aisle

In [117]:
def single_aisle_batching(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running SA Rules")
    
    # Get OrderIDs where Min_Pick_Aisle is equal to Max_Pick_Aisle
    single_aisle_orders = eligible_orders[eligible_orders['Min_Pick_Aisle'] == eligible_orders['Max_Pick_Aisle']][['OrderID', 'Min_Pick_Aisle', 'Cut_DateTime']]

    batched_info = []

    # Group by Min_Pick_Aisle and process each group
    for pick_aisle, group in single_aisle_orders.groupby('Min_Pick_Aisle'):
        # Get OrderIDs for the current aisle
        #rder_ids = set(group['OrderID'])

        # Sort these orders by Cut_DateTime
        sorted_unique_orders = group.sort_values(by='Cut_DateTime')['OrderID']

        i = 0
        while i < len(sorted_unique_orders):
            # Determine the maximum batch size
            if len(sorted_unique_orders[i:]) >= max_containers_per_batch:
                max_batch_size = max_containers_per_batch
            elif len(sorted_unique_orders[i:]) >= (max_containers_per_batch // 2):
                max_batch_size = max_containers_per_batch // 2
            else:
                break

            batch_order_ids = sorted_unique_orders.iloc[i:i + max_batch_size].values

            if len(batch_order_ids) >= max_containers_per_batch / 2 and remaining_containers >= len(batch_order_ids):
                remaining_containers -= len(batch_order_ids)
                batch_id = batch_id_start
                batch_id_start += 1
                batch_rule = 'Single Aisle'
                batch_rule_description = f'Single Aisle - {pick_aisle}'
                #print(f"Creating Batch ID {batch_id} with Orders: {batch_order_ids} using Rule Single-Aisle")

                batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

            i += max_batch_size

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 2 - Single Zone

In [118]:
def single_zone_batching(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Single Zone Rules")

    batched_info = []

    for aisle_per_zone in range(2, max_aisles_per_zone + 1):
        #print(f"Processing for zones of {aisle_per_zone} aisles")
                
        for start_aisle in range(min_aisle, max_aisle - aisle_per_zone + 2, aisle_per_zone):
            end_aisle = start_aisle + aisle_per_zone - 1

            #if start_aisle == 47 and end_aisle == 49:
                #print(f"Processing Zone: Aisles {start_aisle} to {end_aisle}")

            # Filter eligible orders for the current dynamic zone
            eligible_zone_orders = eligible_orders[
                (eligible_orders['BatchedFlag'] == 0) &
                (eligible_orders['Min_Pick_Aisle'] >= start_aisle) &
                (eligible_orders['Max_Pick_Aisle'] <= end_aisle)
            ]

            #if start_aisle == 47 and end_aisle == 49:
                #print(f"Number of eligible orders in zone {start_aisle}-{end_aisle}: {len(eligible_zone_orders)}")

            # Sort orders by Cut_DateTime
            sorted_unique_orders = eligible_zone_orders.sort_values(by='Cut_DateTime')['OrderID']

            # Create batches with max_containers_per_batch
            for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
                potential_batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch]

                if len(potential_batch_order_ids) == max_containers_per_batch:
                    batch_containers = len(potential_batch_order_ids)
                    remaining_containers -= batch_containers

                    batch_id = batch_id_start
                    batch_id_start += 1
                    batch_rule = 'Single Zone'
                    batch_rule_description = f'Single Zone - {aisle_per_zone} Aisles/Zone'
                    batched_info.append((batch_id, potential_batch_order_ids, batch_rule, batch_rule_description))
                    update_batching_info(output_order_data, eligible_orders, potential_batch_order_ids, batch_id, batch_rule, batch_rule_description)
                    #print(f"Creating Batch ID {batch_id} for Zone {start_aisle}-{end_aisle} with Orders: {potential_batch_order_ids.values} using Rule: {batch_rule_description}")
                if remaining_containers <= 0 or i + max_containers_per_batch >= len(sorted_unique_orders):
                    break

            if remaining_containers <= 0:
                break

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 3 - Same Start Zone End Aisle

In [119]:
#To be revised -- ignore for now
def same_start_zone_end_aisle(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Same Start Zone, End Aisle Rules")

    # Precompute min and max pick aisles for each order
    #min_max_aisles = output_order_data.groupby('OrderID')['Pick_Aisle'].agg(['min', 'max'])

    batched_info = []

    # Iterate over each possible end aisle
    for end_aisle_candidate in range(min_aisle, max_aisle + 1):
        # Filter eligible orders with this end aisle and haven't been batched yet
        candidate_order_ids = eligible_orders[
            (eligible_orders['Max_Pick_Aisle'] == end_aisle_candidate) &
            (eligible_orders['BatchedFlag'] == 0)
        ]

        if len(candidate_order_ids) < max_containers_per_batch:
            continue  # Skip if not enough orders for this end aisle

        # Iterate through zones of different sizes
        for aisle_per_zone in range(2, max_aisles_per_zone + 1):
            for start_aisle in range(min_aisle, end_aisle_candidate - aisle_per_zone + 2, aisle_per_zone):
                
                end_aisle = start_aisle + aisle_per_zone - 1
                
                # Filter orders for the current dynamic zone
                eligible_zone_orders = candidate_order_ids[
                    candidate_order_ids['Min_Pick_Aisle'].between(start_aisle, end_aisle)
                ]
                #print(f"End Aisle Candidate: {end_aisle_candidate}, Start Aisle: {start_aisle},  End Aisle: {end_aisle}, OrderCount: {len(eligible_zone_orders)}")
                
                if len(eligible_zone_orders) < max_containers_per_batch:
                    continue  # Skip if not enough orders for this zone
                
                # Sort orders by Cut_DateTime
                sorted_unique_orders = eligible_orders[(eligible_orders['BatchedFlag'] == 0) & eligible_orders['OrderID'].isin(eligible_zone_orders['OrderID'])
                ].sort_values(by='Cut_DateTime')['OrderID']
                
                #print(f"End Aisle Candidate: {end_aisle_candidate}, Start Aisle: {start_aisle},  End Aisle: {end_aisle}, OrderCount: {len(sorted_unique_orders)}")
                
                for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
                    batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values

                    if len(batch_order_ids) != max_containers_per_batch or remaining_containers <= 0:
                        break  # Stop batching if not enough orders or containers

                    remaining_containers -= len(batch_order_ids)
                    batch_id = batch_id_start
                    batch_id_start += 1
                    #print(f"Creating Batch ID {batch_id} with Orders: {batch_order_ids}")

                    batch_rule = 'Same Start Zone End Aisle' 
                    batch_rule_description = f"Same Start Zone ({aisle_per_zone} A/Z), End Aisle {end_aisle_candidate}"
                    batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                    update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 4 - Same Start Zone End Zone

In [120]:
def same_start_zone_end_zone(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Same Start Zone, End Zone Rules")

    batched_info = []

    # Precompute min and max pick aisles for each order
    
    #min_max_aisles = eligible_zone_orders.groupby('OrderID')['Pick_Aisle'].agg(['min', 'max']) #Replace with eligible orders

    for zone_size in range(2, max_aisles_per_zone + 1):
        for start_aisle in range(min_aisle, max_aisle - zone_size + 2, zone_size):
            start_zone_end_aisle = start_aisle + zone_size - 1
            ## yvadalia -- please validate this 
            for end_zone_start_aisle in range(start_aisle + zone_size, max_aisle - zone_size + 2, zone_size):
                end_zone_end_aisle = end_zone_start_aisle + zone_size - 1

                eligible_zone_orders = eligible_orders[
                    (eligible_orders['BatchedFlag'] == 0) &
                    (eligible_orders['Min_Pick_Aisle'].between(start_aisle, start_zone_end_aisle)) &
                    (eligible_orders['Max_Pick_Aisle'].between(end_zone_start_aisle, end_zone_end_aisle))
                ]
                    
                if len(eligible_zone_orders) < max_containers_per_batch:
                    continue
                
                sorted_unique_orders = eligible_orders[(eligible_orders['BatchedFlag'] == 0) & eligible_orders['OrderID'].isin(eligible_zone_orders['OrderID'])
                ].sort_values(by='Cut_DateTime')['OrderID']
                
                #sorted_unique_orders = output_order_data[output_order_data['OrderID'].isin(eligible_zone_orders['OrderID'])]['OrderID'].drop_duplicates().sort_values(by='Cut_DateTime')
                
                # Process batches sequentially
                for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
                    batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values

                    if len(batch_order_ids) != max_containers_per_batch or remaining_containers <= 0:
                        break  # Stop batching if not enough orders or containers

                    remaining_containers -= len(batch_order_ids)
                    batch_id = batch_id_start
                    batch_id_start += 1
                    batch_rule = 'Same Start Zone End Zone' 
                    batch_rule_description = f"Same Start Zone ({zone_size} A/Z) {start_aisle}-{start_zone_end_aisle}, End Zone ({zone_size} A/Z) {end_zone_start_aisle}-{end_zone_end_aisle}"
                    batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                    update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

                if remaining_containers <= 0:
                    break  # Early exit if no more containers

            if remaining_containers <= 0:
                break  # Early exit if no more containers

        if remaining_containers <= 0:
            break  # Early exit if no more containers

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 5 - Legacy Single Zone

In [121]:
def legacy_single_zone_batching(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Legacy Single Zone Rules")

    batched_info = []
    
    sorted_print_zones = sorted(output_order_data['Print_Zone'].unique())
    
    # Iterate through each Print_Zone
    for print_zone in sorted_print_zones:
        #print(f"Processing Print Zone: {print_zone}")

        # Find orders that have lines in the current Print_Zone
        orders_in_print_zone = output_order_data[output_order_data['Print_Zone'] == print_zone]['OrderID'].unique()
        # Further filter to include only those orders where all lines are in the same Print_Zone
        eligible_zone_orders = eligible_orders[
            eligible_orders['OrderID'].isin(orders_in_print_zone) &
            eligible_orders['OrderID'].apply(
                lambda x: output_order_data[output_order_data['OrderID'] == x]['Print_Zone'].nunique() == 1
            ) & (eligible_orders['BatchedFlag'] == 0)
        ]
        
        sorted_unique_orders = eligible_orders[(eligible_orders['BatchedFlag'] == 0) & eligible_orders['OrderID'].isin(eligible_zone_orders['OrderID'])
                ].sort_values(by='Cut_DateTime')['OrderID']
        #sorted_unique_orders = output_order_data[output_order_data['OrderID'].isin(eligible_zone_orders['OrderID'])].drop_duplicates('OrderID').sort_values(by='Cut_DateTime')['OrderID']

        for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
            batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values
            if len(batch_order_ids) == max_containers_per_batch:
                batch_containers = len(batch_order_ids)
                remaining_containers -= batch_containers
                batch_id = batch_id_start
                batch_id_start += 1
                batch_rule = 'Legacy Single Zone' 
                batch_rule_description = f"Legacy Single Zone - Print Zone {print_zone}"
                batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

            if remaining_containers <= 0:
                break  # Early exit if no more containers

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 6 - Legacy Same Start Zone End Aisle

In [122]:
def legacy_same_start_zone_end_aisle(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Legacy Same Start Zone, End Aisle Rules")

    batched_info = []
    sorted_print_zones = sorted(output_order_data['Print_Zone'].unique())
    
    # Precompute Min and Max Aisles for each order
    min_print_zone = output_order_data.groupby('OrderID')['Print_Zone'].min()
    max_aisles_per_order = output_order_data.groupby('OrderID')['Pick_Aisle'].max()

    for print_zone in sorted_print_zones:
        # Filter orders with Min_Pick_Aisle in the current Print_Zone
        eligible_zone_orders = eligible_orders[
            (eligible_orders['OrderID'].isin(min_print_zone[min_print_zone == print_zone].index)) &
            (eligible_orders['BatchedFlag'] == 0)
        ]

        if len(eligible_zone_orders) < max_containers_per_batch:
            continue  # Skip if no orders are eligible

        for end_aisle_candidate in range(min_aisle, max_aisle + 1):
            # Filter orders with the same max aisle (end_aisle_candidate)
            candidate_order_ids = eligible_zone_orders[ eligible_zone_orders['OrderID'].isin(
                max_aisles_per_order[max_aisles_per_order == end_aisle_candidate].index)]

            if candidate_order_ids.empty:
                continue  # Skip if no orders are eligible
            
            sorted_unique_orders = candidate_order_ids[candidate_order_ids['BatchedFlag'] == 0].sort_values(by='Cut_DateTime')['OrderID']
            #sorted_unique_orders = output_order_data[output_order_data['OrderID'].isin(eligible_zone_orders['OrderID'])]['OrderID'].drop_duplicates().sort_values(by='Cut_DateTime')

            for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
                batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values

                if len(batch_order_ids) != max_containers_per_batch or remaining_containers <= 0:
                    break  # Stop batching if not enough orders or containers

                remaining_containers -= len(batch_order_ids)
                batch_id = batch_id_start
                batch_id_start += 1
                batch_rule = 'Legacy Same Start Zone End Aisle' 
                batch_rule_description = f"Legacy Same Start Zone (Print Zone {print_zone}), End Aisle {end_aisle_candidate}"
                batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

                if remaining_containers <= 0:
                    break  # Early exit if no more containers

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 7 - Legacy Same Start Zone End Zone

In [123]:
def legacy_same_start_zone_end_zone(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Legacy Same Start Zone, End Zone Rules")

    batched_info = []

    # Precompute Min and Max Print_Zones for each order
    min_zones_per_order = output_order_data.groupby('OrderID')['Print_Zone'].min()
    max_zones_per_order = output_order_data.groupby('OrderID')['Print_Zone'].max()

    sorted_print_zones = sorted(output_order_data['Print_Zone'].unique())

    for start_zone in sorted_print_zones:
        for end_zone in sorted_print_zones:
            if start_zone > end_zone:  # Skip invalid combinations
                continue

            # Filter orders with both start and end print zones
            eligible_zone_orders = eligible_orders[
                eligible_orders['OrderID'].isin(min_zones_per_order[min_zones_per_order == start_zone].index) &
                eligible_orders['OrderID'].isin(max_zones_per_order[max_zones_per_order == end_zone].index) &
                (eligible_orders['BatchedFlag'] == 0)
            ]
            #print(eligible_zone_orders)
            if len(eligible_zone_orders) < max_containers_per_batch or remaining_containers <= 0:
                continue  # Skip if no orders are eligible or no containers left
            
            sorted_unique_orders = eligible_orders[(eligible_orders['BatchedFlag'] == 0) & eligible_orders['OrderID'].isin(eligible_zone_orders['OrderID'])
                ].sort_values(by='Cut_DateTime')['OrderID'] 
            #sorted_unique_orders = output_order_data[output_order_data['OrderID'].isin(eligible_zone_orders['OrderID'])]['OrderID'].drop_duplicates().sort_values(by='Cut_DateTime')

            for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
                batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values
                #print(batch_order_ids)
                if len(batch_order_ids) != max_containers_per_batch:
                    break  # Stop batching if not enough orders

                remaining_containers -= len(batch_order_ids)
                batch_id = batch_id_start
                batch_id_start += 1
                batch_rule = 'Legacy Same Start Zone End Zone' 
                batch_rule_description = f"Legacy Same Start Zone ({start_zone}), End Zone ({end_zone})"
                batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
                update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

                if remaining_containers <= 0:
                    break  # Early exit if no more containers

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 8 - Legacy Same Start Zone

In [124]:
def legacy_same_start_zone(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running Legacy Same Start Zone Rules")

    batched_info = []

    # Precompute Min Print_Zone for each order
    min_zones_per_order = output_order_data.groupby('OrderID')['Print_Zone'].min()

    # Iterate through each unique Print_Zone
    sorted_print_zones = sorted(output_order_data['Print_Zone'].unique())

    for print_zone in sorted_print_zones:
        # Select orders that start in the current Print_Zone and are not yet batched
        eligible_zone_order_ids = eligible_orders[
            eligible_orders['OrderID'].isin(min_zones_per_order[min_zones_per_order == print_zone].index) &
            (eligible_orders['BatchedFlag'] == 0)
        ]['OrderID']

        if len(eligible_zone_order_ids) < max_containers_per_batch or remaining_containers <= 0:
            continue  # Skip if no orders are eligible or no containers left

        # Sort orders by Cut_DateTime
        sorted_unique_orders = eligible_orders[
            (eligible_orders['OrderID'].isin(eligible_zone_order_ids)) &
            (eligible_orders['BatchedFlag'] == 0)
        ].sort_values(by='Cut_DateTime')['OrderID']

        for i in range(0, len(sorted_unique_orders), max_containers_per_batch):
            batch_order_ids = sorted_unique_orders.iloc[i:i + max_containers_per_batch].values

            if len(batch_order_ids) != max_containers_per_batch:
                break  # Stop batching if not enough orders


            remaining_containers -= len(batch_order_ids)
            batch_id = batch_id_start
            batch_id_start += 1
            batch_rule = 'Legacy Same Start Zone' 
            batch_rule_description = f"Legacy Same Start Zone (Print Zone {print_zone})"
            batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
            update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

            if remaining_containers <= 0:
                break  # Early exit if no more containers

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Rule 9 - All Pickables

In [125]:
def all_pickables(current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, batch_id_start, max_aisles_per_zone, remaining_containers):
    #print("Running All Pickables Batching")

    batched_info = []

    # Directly get unbatched orders sorted by Cut_DateTime
    unbatched_orders = eligible_orders.loc[eligible_orders['BatchedFlag'] == 0, 'OrderID']
    sorted_unique_orders = output_order_data.loc[output_order_data['OrderID'].isin(unbatched_orders)].drop_duplicates('OrderID').sort_values(by='Cut_DateTime')['OrderID']

    if sorted_unique_orders.empty:
        return batched_info, output_order_data, batch_id_start, remaining_containers

    i = 0
    while i < len(sorted_unique_orders) and remaining_containers > 0:
        # Calculate the potential batch size (either max_containers_per_batch or remaining orders)
        potential_batch_size = int(min(max_containers_per_batch, len(sorted_unique_orders) - i, remaining_containers))
        batch_order_ids = sorted_unique_orders.iloc[i:i + potential_batch_size].values

        if len(batch_order_ids) > 0:  # Check to ensure there are orders to batch
            remaining_containers -= len(batch_order_ids)
            batch_id = batch_id_start
            batch_id_start += 1
            batch_rule = 'All Pickables'
            batch_rule_description = 'All Pickables'
            batched_info.append((batch_id, batch_order_ids, batch_rule, batch_rule_description))
            update_batching_info(output_order_data, eligible_orders, batch_order_ids, batch_id, batch_rule, batch_rule_description)

        i += potential_batch_size

    return batched_info, output_order_data, batch_id_start, remaining_containers


## Utility Functions

## Update Batching Info

In [126]:
# Function to update output_order_data and eligible_orders
def update_batching_info(output_order_data, eligible_orders, order_ids, batch_id, batch_rule, batch_rule_description):
    output_order_data.loc[output_order_data['OrderID'].isin(order_ids), ['BatchedFlag', 'BatchID', 'Rule', 'Rule_Desc']] = \
        [1, batch_id, batch_rule, batch_rule_description]
    eligible_orders.loc[eligible_orders['OrderID'].isin(order_ids), 'BatchedFlag'] = 1


## Update Order Master post-batching

In [127]:
# Function to update order_master after each batching rule
def update_order_master_after_batching(batched_info, rule_description, current_date_hour_dt):
    for batch_id, batch_order_ids, batch_rule, batch_rule_description in batched_info:
        if len(batch_order_ids) != len(set(batch_order_ids)):
            print(f"Duplicate OrderIDs found in Batch ID {batch_id}: {batch_order_ids}: Rule: {batch_rule} ")
        update_mask = order_master['OrderID'].isin(batch_order_ids)
        order_master.loc[update_mask, 'BatchedFlag'] = 1
        order_master.loc[update_mask, 'BatchID'] = batch_id
        order_master.loc[update_mask, 'Rule'] = batch_rule
        order_master.loc[update_mask, 'Rule_Description'] = batch_rule_description
        order_master.loc[update_mask, 'BatchDate'] = current_date_hour_dt.date()
        order_master.loc[update_mask, 'BatchHour'] = current_date_hour_dt.hour
        # Only update Batch_Date_Hour for the updated rows
        order_master.loc[update_mask, 'Batch_Date_Hour'] = pd.to_datetime(order_master.loc[update_mask, 'BatchDate'].astype(str) + ' ' + order_master.loc[update_mask, 'BatchHour'].astype(str) + ':00:00')


In [128]:
def create_datetime(row):
    if pd.notnull(row['BatchDate']) and pd.notnull(row['BatchHour']):
        return pd.to_datetime(f"{row['BatchDate']} {row['BatchHour']}:00:00")
    else:
        return pd.NaT


## Run AutoBatching Sim

## Set Simulation Parameters

In [129]:
# Simulation Parameters
total_runtime_hours = 1
max_containers_per_batch = 20
max_aisles_per_zone = 8
batch_id_start = 1


In [130]:
## Ignore warnings
import warnings; warnings.simplefilter('ignore')


In [131]:

# Running the Simulation
min_date_hour = '2025-03-06 14:00:00' #order_master['Arrive_Date_Hour'].min()
end_time = '2025-03-06 15:00:00' #order_master['Arrive_Date_Hour'].max() + timedelta(hours=1) # min_date_hour + timedelta(hours=total_runtime_hours-1)

# Process each batching rule
batching_rules = [
    ("Single Aisle", single_aisle_batching),
    ("Single Zone", single_zone_batching),
    ("Same Start Zone, End Aisle", same_start_zone_end_aisle),
    ("Same Start Zone, End Zone", same_start_zone_end_zone),
    ("Legacy Single Zone", legacy_single_zone_batching),
    ("Legacy Same Start Zone, End Aisle", legacy_same_start_zone_end_aisle),
    ("Legacy Same Start Zone, End Zone", legacy_same_start_zone_end_zone),
    ("Legacy Same Start Zone", legacy_same_start_zone),
    ("All Pickables", all_pickables)
]

# Running the Simulation
for current_date_hour in tqdm(pd.date_range(start=min_date_hour, end=end_time, freq='H')):
    print(f"Processing for Date_Hour: {current_date_hour}")
    current_date_hour_dt = pd.to_datetime(current_date_hour)

    # Find the container target for the current hour
    target_row = container_targets_df[container_targets_df['Date_Hour'] == current_date_hour]
    target_containers_per_hour = target_row['Containers_Target'].iloc[0] if not target_row.empty else 0
    remaining_containers = target_containers_per_hour

    # Initialize execution time tracking for each rule
    execution_times = {rule: 0 for rule, _ in batching_rules}
    execution_counts = {rule: 0 for rule, _ in batching_rules}

    for rule_description, batching_function in batching_rules:
        start_time = time.time()
        eligible_orders = order_master[(order_master['Arrive_Date_Hour'] <= current_date_hour) & (order_master['BatchedFlag'] == 0)]
        min_aisle = eligible_orders['Min_Pick_Aisle'].min()
        max_aisle = eligible_orders['Max_Pick_Aisle'].max()

        if not eligible_orders.empty and remaining_containers > 0:
            batched_info, output_order_data, batch_id_start, remaining_containers = batching_function(
                current_date_hour_dt, eligible_orders, output_order_data, max_containers_per_batch, 
                batch_id_start, max_aisles_per_zone, remaining_containers
            )
            if batched_info:
                update_order_master_after_batching(batched_info, rule_description, current_date_hour_dt)
        
        end_time = time.time()
        execution_times[rule_description] += (end_time - start_time)
        execution_counts[rule_description] += 1

    # Update Hourly Statistics
    unique_batches = set(order_master[(order_master['BatchDate'] == current_date_hour.date()) & (order_master['BatchHour'] == current_date_hour.hour)]['BatchID'].dropna())
    unique_orders = set(order_master[(order_master['BatchDate'] == current_date_hour.date()) & (order_master['BatchHour'] == current_date_hour.hour)]['OrderID'].dropna())
    sum_of_units = order_master[(order_master['BatchDate'] == current_date_hour.date()) & (order_master['BatchHour'] == current_date_hour.hour)]['Total_Pick_Quantity'].sum()

    new_row = {
        'Date': current_date_hour.date(), 'Hour': current_date_hour.hour, 
        'Date_Hour': current_date_hour, 'Unique_Batch_ID_Count': len(unique_batches), 
        #'Unique_Order_Count': len(unique_orders),
        'Target_Containers': target_containers_per_hour, 
        'Containers_Batched': target_containers_per_hour - remaining_containers, 'Sum_of_Units_Batched': sum_of_units
    }
    hourly_stats.loc[len(hourly_stats)] = new_row

# Calculate and display average execution times
for rule, total_time in execution_times.items():
    average_time = total_time / execution_counts[rule] if execution_counts[rule] > 0 else 0
    print(f"Average execution time for {rule}: {average_time:.2f} seconds")


  0%|          | 0/2 [00:00<?, ?it/s]

Processing for Date_Hour: 2025-03-06 14:00:00
Processing for Date_Hour: 2025-03-06 15:00:00
Average execution time for Single Aisle: 0.00 seconds
Average execution time for Single Zone: 0.00 seconds
Average execution time for Same Start Zone, End Aisle: 0.00 seconds
Average execution time for Same Start Zone, End Zone: 0.00 seconds
Average execution time for Legacy Single Zone: 0.00 seconds
Average execution time for Legacy Same Start Zone, End Aisle: 0.00 seconds
Average execution time for Legacy Same Start Zone, End Zone: 0.00 seconds
Average execution time for Legacy Same Start Zone: 0.00 seconds
Average execution time for All Pickables: 0.00 seconds


In [132]:
hourly_stats


,Date,Hour,Date_Hour,Unique_Batch_ID_Count,Target_Containers,Containers_Batched,Sum_of_Units_Batched
0,2025-03-06,14,2025-03-06 14:00:00,69,13300,1380,6029
1,2025-03-06,15,2025-03-06 15:00:00,0,0,0,0


In [133]:
output_order_data['Batch_Date_Hour'] = output_order_data['OrderID'].map(order_master.set_index('OrderID')['Batch_Date_Hour'])
#output_order_data.to_excel("C:/Users/vpandav/OneDrive - Chewy.com, LLC/Simulation/Pack Mule/Pack Mule Simulation Iter4/Safety_Scenario/20_container_cart.xlsx", index=False)


In [134]:
output_order_data


,OrderID,Arrival_Date,Arrive_Date,Arrive_Hour,Available_To_Batch,Cut_DateTime,SKU,uom_weight,Pick_Quantity,Pick_Location,Print_Zone,Pick_Aisle,Picking_Flow_as_int,BatchedFlag,BatchID,Arrive_Date_Hour,Available_To_Batch_Date_Hour,Rule,Rule_Desc,Batch_Date_Hour
21,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,178721,1.15,1,F-44-A01,2,9,23436,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
22,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,178723,1.25,1,20-46-A01,3,22,205528,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
23,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,217436,5.25,1,06-48-A01,2,15,73140,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
46,1062803566,2025-03-04,2025-03-04 00:56:00,0,1,2025-03-06 15:00:00,75964,5.45,1,S5-04-A01,1,4,2634,1,64,2025-03-04 00:00:00,2025-03-04 01:00:00,Legacy Same Start Zone,Legacy Same Start Zone (Print Zone 1),2025-03-06 14:00:00
47,1062803566,2025-03-04,2025-03-04 00:56:00,0,1,2025-03-06 15:00:00,58936,6.36,1,02-49-A02,2,13,57233,1,64,2025-03-04 00:00:00,2025-03-04 01:00:00,Legacy Same Start Zone,Legacy Same Start Zone (Print Zone 1),2025-03-06 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21495,1064096180,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 18:00:00,68194,0.80,2,60-28-A01,5,43,394272,1,45,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (4 A/Z), End Aisle 43",2025-03-06 14:00:00
21496,1064096180,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 18:00:00,277178,2.24,1,36-26-C04,4,30,280725,1,45,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (4 A/Z), End Aisle 43",2025-03-06 14:00:00
21539,1064096311,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 15:00:00,105151,0.20,1,K-39-C03,2,11,40909,1,13,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 21",2025-03-06 14:00:00
21540,1064096311,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 15:00:00,165957,0.85,1,14-47-C12,3,19,183471,1,13,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 21",2025-03-06 14:00:00


In [135]:
# Group by 'BatchRule' and count unique 'BatchID' and 'OrderID'
rules_summary = order_master.groupby('Rule').agg({
    'BatchID': pd.Series.nunique,
    'OrderID': pd.Series.nunique
}).rename(columns={'BatchID': 'Unique_Batch_ID_Count', 'OrderID': 'Unique_Order_Count'}).reset_index()

# Print the summary
rules_summary


,Rule,Unique_Batch_ID_Count,Unique_Order_Count
0,All Pickables,2,40
1,Legacy Same Start Zone,4,80
2,Legacy Same Start Zone End Zone,2,40
3,Same Start Zone End Aisle,49,980
4,Same Start Zone End Zone,12,240


## Prepare FlexSim Input File

In [136]:
container_targets_df['FlexSim_Time'] = container_targets_df.apply(lambda x: ((24*(x['batch_date']-container_targets_df['batch_date'].min()).days) + x['batch_hour']) * 3600, axis=1)  
container_targets_df


,wh_id,batch_date,batch_hour,sum,Containers_Target,containers_released,Date_Hour,FlexSim_Time
0,AVP1,2025-03-06,14,70,13300,1474,2025-03-06 14:00:00,50400


In [137]:
# Step 1: Create a copy of output_order_data
flexsim_input_df = output_order_data.copy()

# Step 2: Update Batch_Date_Hour in the new DataFrame
flexsim_input_df['Batch_Date_Hour'] = flexsim_input_df['OrderID'].map(order_master.set_index('OrderID')['Batch_Date_Hour'])

# Step 3: Match Batch_Date_Hour with Date_Hour in container_targets_df to find FlexSim_Time
# Ensure both columns are in the same datetime format
container_targets_df['Date_Hour'] = pd.to_datetime(container_targets_df['Date_Hour'])
flexsim_input_df['Batch_Date_Hour'] = pd.to_datetime(flexsim_input_df['Batch_Date_Hour'])

# Map FlexSim_Time from container_targets_df to flexsim_input_df
flexsim_input_df['FlexSim_Time'] = flexsim_input_df['Batch_Date_Hour'].map(container_targets_df.set_index('Date_Hour')['FlexSim_Time'])

# Step 4: Keep only specified columns
flexsim_input_df = flexsim_input_df[['FlexSim_Time', 'BatchID', 'OrderID', 'SKU', 'Pick_Quantity', 'Pick_Location', 'Picking_Flow_as_int']]

# Drop any rows with missing values, if necessary
flexsim_input_df.dropna(inplace=True)

flexsim_input_df = flexsim_input_df.sort_values(by=['FlexSim_Time', 'BatchID', 'Picking_Flow_as_int'], ascending=True)


In [138]:
output_order_data


,OrderID,Arrival_Date,Arrive_Date,Arrive_Hour,Available_To_Batch,Cut_DateTime,SKU,uom_weight,Pick_Quantity,Pick_Location,Print_Zone,Pick_Aisle,Picking_Flow_as_int,BatchedFlag,BatchID,Arrive_Date_Hour,Available_To_Batch_Date_Hour,Rule,Rule_Desc,Batch_Date_Hour
21,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,178721,1.15,1,F-44-A01,2,9,23436,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
22,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,178723,1.25,1,20-46-A01,3,22,205528,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
23,1062781132,2025-03-04,2025-03-04 00:32:00,0,1,2025-03-06 18:00:00,217436,5.25,1,06-48-A01,2,15,73140,1,17,2025-03-04 00:00:00,2025-03-04 01:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 22",2025-03-06 14:00:00
46,1062803566,2025-03-04,2025-03-04 00:56:00,0,1,2025-03-06 15:00:00,75964,5.45,1,S5-04-A01,1,4,2634,1,64,2025-03-04 00:00:00,2025-03-04 01:00:00,Legacy Same Start Zone,Legacy Same Start Zone (Print Zone 1),2025-03-06 14:00:00
47,1062803566,2025-03-04,2025-03-04 00:56:00,0,1,2025-03-06 15:00:00,58936,6.36,1,02-49-A02,2,13,57233,1,64,2025-03-04 00:00:00,2025-03-04 01:00:00,Legacy Same Start Zone,Legacy Same Start Zone (Print Zone 1),2025-03-06 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21495,1064096180,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 18:00:00,68194,0.80,2,60-28-A01,5,43,394272,1,45,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (4 A/Z), End Aisle 43",2025-03-06 14:00:00
21496,1064096180,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 18:00:00,277178,2.24,1,36-26-C04,4,30,280725,1,45,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (4 A/Z), End Aisle 43",2025-03-06 14:00:00
21539,1064096311,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 15:00:00,105151,0.20,1,K-39-C03,2,11,40909,1,13,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 21",2025-03-06 14:00:00
21540,1064096311,2025-03-06,2025-03-06 13:53:00,13,14,2025-03-06 15:00:00,165957,0.85,1,14-47-C12,3,19,183471,1,13,2025-03-06 13:00:00,2025-03-06 14:00:00,Same Start Zone End Aisle,"Same Start Zone (2 A/Z), End Aisle 21",2025-03-06 14:00:00


In [139]:
# Filter out rows where BatchID is 0 or NaN
filtered_data = output_order_data[output_order_data['BatchID'].notna() & (output_order_data['BatchID'] != 0)]

# Total Tours: distinct count of BatchID
total_tours = filtered_data['BatchID'].nunique()

# Sum of Unique Aisles: sum of distinct Pick_Aisle for each BatchID
unique_aisles_sum = filtered_data.groupby('BatchID')['Pick_Aisle'].nunique().sum()

# Sum of Aisle Span: sum of max(Pick_Aisle) - min(Pick_Aisle) for each BatchID
aisle_span_sum = filtered_data.groupby('BatchID').apply(lambda x: x['Pick_Aisle'].max() - x['Pick_Aisle'].min()).sum()

# Sum of Units: sum of PickQuantity where BatchID is not 0 or NaN
units_sum = filtered_data['Pick_Quantity'].sum()

# Print the results
print(f"Total Tours: {total_tours}")
print(f"Sum of Unique Aisles: {unique_aisles_sum}")
print(f"Sum of Aisle Span: {aisle_span_sum}")
print(f"Sum of Units: {units_sum}")


Total Tours: 69
Sum of Unique Aisles: 900
Sum of Aisle Span: 1091
Sum of Units: 6029
